In [1]:
import requests as rq
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from time import sleep

In [2]:
def click_show_more(driver):
    try:
        show_more_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="c8hZ6"]/div/div'))
        )

        # Scroll to the "Show More" button to make it clickable
        ActionChains(driver).move_to_element(show_more_button).perform()
        time.sleep(1)  # Add a small delay to ensure the button is clickable

        # Click the "Show More" button
        show_more_button.click()

        # You may need to wait for the new content to load before proceeding
        WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.XPATH, 'loading_indicator_xpath')))
        
    except Exception as e:
        print(f"An error occurred while clicking 'Show More': {e}")


In [3]:
sources=['CMN','PAR','ROM','IST']
destinations=['CMN','PAR','ROM','IST']
classes=['economy','business']
Dates_1=['2023-12-09','2024-01-12','2024-02-10','2024-03-17','2024-04-18','2024-05-29','2024-06-01']
Dates_2=['2024-07-04','2024-08-16','2024-09-03','2024-10-19','2024-11-05']

# Features

In [4]:
def get_airline(soup):
    #get airlines
    airlines=[]
    items=soup.find_all('div',class_='J0g6-operator-text') 
    for item in items :
        airl=item.text
        airl = airl.split(',')[0].strip()
        airlines.append(airl)
    
    return airlines

In [5]:
def get_duration(soup):
    #getduration
    durations=[]
    duration_div=soup.find_all('div',class_='xdW8 xdW8-mod-full-airport')
    for d in duration_div :
        duration=d.find('div',class_='vmXl vmXl-mod-variant-default').text
        durations.append(duration)
    return durations

In [6]:
def get_stops(soup):
    stops=[]
    stop_div=soup.find_all('div',class_='vmXl vmXl-mod-variant-default')
    for s in stop_div :
        selected_element = s.select_one('.JWEO-stops-text, .JWEO-mod-warning')
        if selected_element:
          stops.append(selected_element.text)

    return stops

In [7]:
def get_price(soup):   
    #getprice
    prices=[]
    price_div=soup.find_all('div',class_='f8F1-price-text-container')

    for p in price_div :
        price=p.find('div',class_='f8F1-price-text').text
        price= price.replace('\xa0', ' ')
        prices.append(price)
    return prices

In [8]:
def get_class(soup):
    #getclass
    classes=[]
    class_div=soup.find_all('div',class_='aC3z-option')
    for c in class_div :
        classe=c.find('div',class_='aC3z-name').text
        classes.append(classe)
    return classes

In [9]:
#get_depature_time
def get_dep_time(soup):
    deps=[]
    dep_div=soup.find_all('div',class_='vmXl vmXl-mod-variant-large')
    for d in dep_div:
        depature=d.find_all('span')
        deps.append(depature[0].text)
    return deps
#get_arrival_time
def get_arr_time(soup):
    arrs=[]
    arr_div=soup.find_all('div',class_='vmXl vmXl-mod-variant-large')
    for a in arr_div:
        arrival=a.find_all('span')
        arrs.append(arrival[2].text)
    return arrs


# Scraping economy class

In [33]:
df=pd.DataFrame(columns=["Airline", "Source", "Destination","Duration","stops","class","depature time","arrival time", "Price","Date"])
for k in range(len(Dates_1)):
    for j in range(len(classes)):    
        for i in range(len(sources)): 
            for l in range(len(destinations)):
                if sources[i]==destinations[l]:
                    continue
                driver = webdriver.Edge()
                kayak=f"https://www.kayak.ae/flights/{sources[i]}-{destinations[l]}/{Dates_1[k]}/{classes[j]}?sort=bestflight_a"
                print(Dates_1[k])
                print(sources[i])
                print(destinations[l])
                print(classes[j])
                driver.get(kayak)
                # Click "Show More" button
                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'ULvh-button.show-more-button')))
                '''count=2
                while count>1:
                    try:
                        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'ULvh-button.show-more-button'))).click()
                        print("MORE button clicked")
                    except TimeoutException:
                        break
                    count-=1
                    sleep(5)'''

                soup = BeautifulSoup(driver.page_source, 'html.parser')
                airlines = get_airline(soup)
                print(len(airlines))
                stops = get_stops(soup)
                print(len(stops))
                dep_time=get_dep_time(soup)
                print(len(dep_time))
                arr_time =get_arr_time(soup)
                print(len(arr_time))
                prices = get_price(soup)
                print(len(prices))
                duration = get_duration(soup)
                print(len(duration))
                df = pd.concat([df,pd.DataFrame({
                                'Airline': airlines,
                                'Duration': duration,
                                'stops' : stops,
                                'class' : classes[j],
                                'Source':sources[i],
                                'Destination': destinations[l],
                                'depature time' : dep_time ,
                                'arrival time' : arr_time,
                                'Price' : prices,
                                'Date' : Dates_1[k]})])


                df = df.replace('\n','', regex=True)
                #df = df.reset_index(drop = True)
                # save data as csv file for each route

                print(f"Succesfully saved {sources[i]} => {destinations[l]} route  ")
                driver.quit()
df.to_csv('../data/airlines_dataset.csv',index=False)
driver.quit()

2023-12-09
CMN
PAR
economy
15
15
15
15
15
15
Succesfully saved CMN => PAR route  
2023-12-09
CMN
ROM
economy
15
15
15
15
15
15
Succesfully saved CMN => ROM route  
2023-12-09
CMN
IST
economy
15
15
15
15
15
15
Succesfully saved CMN => IST route  
2023-12-09
PAR
CMN
economy
15
15
15
15
15
15
Succesfully saved PAR => CMN route  
2023-12-09
PAR
ROM
economy
15
15
15
15
15
15
Succesfully saved PAR => ROM route  
2023-12-09
PAR
IST
economy
15
15
15
15
15
15
Succesfully saved PAR => IST route  
2023-12-09
ROM
CMN
economy
16
16
16
16
16
16
Succesfully saved ROM => CMN route  
2023-12-09
ROM
PAR
economy
15
15
15
15
15
15
Succesfully saved ROM => PAR route  
2023-12-09
ROM
IST
economy
15
15
15
15
15
15
Succesfully saved ROM => IST route  
2023-12-09
IST
CMN
economy
15
15
15
15
15
15
Succesfully saved IST => CMN route  
2023-12-09
IST
PAR
economy
16
16
16
16
16
16
Succesfully saved IST => PAR route  
2023-12-09
IST
ROM
economy
15
15
15
15
15
15
Succesfully saved IST => ROM route  
2023-12-09
CMN
P

16
16
16
16
16
16
Succesfully saved PAR => CMN route  
2024-04-18
PAR
ROM
economy
15
15
15
15
15
15
Succesfully saved PAR => ROM route  
2024-04-18
PAR
IST
economy
15
15
15
15
15
15
Succesfully saved PAR => IST route  
2024-04-18
ROM
CMN
economy
16
16
16
16
16
16
Succesfully saved ROM => CMN route  
2024-04-18
ROM
PAR
economy
15
15
15
15
15
15
Succesfully saved ROM => PAR route  
2024-04-18
ROM
IST
economy
15
15
15
15
15
15
Succesfully saved ROM => IST route  
2024-04-18
IST
CMN
economy
15
15
15
15
15
15
Succesfully saved IST => CMN route  
2024-04-18
IST
PAR
economy
15
15
15
15
15
15
Succesfully saved IST => PAR route  
2024-04-18
IST
ROM
economy
15
15
15
15
15
15
Succesfully saved IST => ROM route  
2024-04-18
CMN
PAR
business
15
15
15
15
15
15
Succesfully saved CMN => PAR route  
2024-04-18
CMN
ROM
business
15
15
15
15
15
15
Succesfully saved CMN => ROM route  
2024-04-18
CMN
IST
business
15
15
15
15
15
15
Succesfully saved CMN => IST route  
2024-04-18
PAR
CMN
business
16
16
16
16


In [ ]:
#raise SystemExit("Stop right there!")

In [10]:
df=pd.read_csv('../data/airlines_dataset.csv')

In [13]:
for k in range(len(Dates_2)):
    for j in range(len(classes)):    
        for i in range(len(sources)): 
            for l in range(len(destinations)):
                if sources[i]==destinations[l]:
                    continue
                driver = webdriver.Edge()
                kayak=f"https://www.kayak.ae/flights/{sources[i]}-{destinations[l]}/{Dates_2[k]}/{classes[j]}?sort=bestflight_a"
                print(Dates_2[k])
                print(sources[i])
                print(destinations[l])
                print(classes[j])
                driver.get(kayak)
                # Click "Show More" button
                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'ULvh-button.show-more-button')))
                '''count=2
                while count>1:
                    try:
                        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'ULvh-button.show-more-button'))).click()
                        print("MORE button clicked")
                    except TimeoutException:
                        break
                    count-=1
                    sleep(5)'''

                soup = BeautifulSoup(driver.page_source, 'html.parser')
                airlines = get_airline(soup)
                print(len(airlines))
                stops = get_stops(soup)
                print(len(stops))
                dep_time=get_dep_time(soup)
                print(len(dep_time))
                arr_time =get_arr_time(soup)
                print(len(arr_time))
                prices = get_price(soup)
                print(len(prices))
                duration = get_duration(soup)
                print(len(duration))
                df = pd.concat([df,pd.DataFrame({
                                'Airline': airlines,
                                'Duration': duration,
                                'stops' : stops,
                                'class' : classes[j],
                                'Source':sources[i],
                                'Destination': destinations[l],
                                'depature time' : dep_time ,
                                'arrival time' : arr_time,
                                'Price' : prices,
                                'Date' : Dates_2[k]})])


                df = df.replace('\n','', regex=True)
                #df = df.reset_index(drop = True)
                # save data as csv file for each route

                print(f"Succesfully saved {sources[i]} => {destinations[l]} route  ")
                driver.quit()
df.to_csv('../data/airlines_dataset.csv',index=False)
driver.quit()

2024-07-04
CMN
PAR
economy
15
15
15
15
15
15
Succesfully saved CMN => PAR route  
2024-07-04
CMN
ROM
economy
15
15
15
15
15
15
Succesfully saved CMN => ROM route  
2024-07-04
CMN
IST
economy
15
15
15
15
15
15
Succesfully saved CMN => IST route  
2024-07-04
PAR
CMN
economy
16
16
16
16
16
16
Succesfully saved PAR => CMN route  
2024-07-04
PAR
ROM
economy
15
15
15
15
15
15
Succesfully saved PAR => ROM route  
2024-07-04
PAR
IST
economy
15
15
15
15
15
15
Succesfully saved PAR => IST route  
2024-07-04
ROM
CMN
economy
16
16
16
16
16
16
Succesfully saved ROM => CMN route  
2024-07-04
ROM
PAR
economy
15
15
15
15
15
15
Succesfully saved ROM => PAR route  
2024-07-04
ROM
IST
economy
15
15
15
15
15
15
Succesfully saved ROM => IST route  
2024-07-04
IST
CMN
economy
15
15
15
15
15
15
Succesfully saved IST => CMN route  
2024-07-04
IST
PAR
economy
15
15
15
15
15
15
Succesfully saved IST => PAR route  
2024-07-04
IST
ROM
economy
15
15
15
15
15
15
Succesfully saved IST => ROM route  
2024-07-04
CMN
P

15
15
15
15
15
15
Succesfully saved PAR => CMN route  
2024-11-05
PAR
ROM
economy
15
15
15
15
15
15
Succesfully saved PAR => ROM route  
2024-11-05
PAR
IST
economy
16
16
16
16
16
16
Succesfully saved PAR => IST route  
2024-11-05
ROM
CMN
economy
15
15
15
15
15
15
Succesfully saved ROM => CMN route  
2024-11-05
ROM
PAR
economy
15
15
15
15
15
15
Succesfully saved ROM => PAR route  
2024-11-05
ROM
IST
economy
16
16
16
16
16
16
Succesfully saved ROM => IST route  
2024-11-05
IST
CMN
economy
15
15
15
15
15
15
Succesfully saved IST => CMN route  
2024-11-05
IST
PAR
economy
15
15
15
15
15
15
Succesfully saved IST => PAR route  
2024-11-05
IST
ROM
economy
15
15
15
15
15
15
Succesfully saved IST => ROM route  
2024-11-05
CMN
PAR
business
15
15
15
15
15
15
Succesfully saved CMN => PAR route  
2024-11-05
CMN
ROM
business
15
15
15
15
15
15
Succesfully saved CMN => ROM route  
2024-11-05
CMN
IST
business


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF710253F02+58082]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF7101E54B2+224082]
	(No symbol) [0x00007FF710009D9B]
	(No symbol) [0x00007FF710059A9B]
	(No symbol) [0x00007FF710059BAC]
	(No symbol) [0x00007FF71009B8C7]
	(No symbol) [0x00007FF71007BF5F]
	(No symbol) [0x00007FF71004D231]
	(No symbol) [0x00007FF710098A03]
	(No symbol) [0x00007FF71007BB83]
	(No symbol) [0x00007FF71004BFB4]
	(No symbol) [0x00007FF71004B405]
	(No symbol) [0x00007FF71004C7E4]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF7103FD87A+1003402]
	(No symbol) [0x00007FF7100D06A9]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF71015A8F5+35045]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF710152A8D+2685]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF7103FC573+998531]
	Microsoft::Applications::Events::ILogConfiguration::operat

In [11]:
 for j in range(len(classes)):    
        for i in range(len(sources)): 
            for l in range(len(destinations)):
                if sources[i]==destinations[l]:
                    continue
                driver = webdriver.Edge()
                kayak=f"https://www.kayak.ae/flights/{sources[i]}-{destinations[l]}/2024-11-05/{classes[j]}?sort=bestflight_a"
                print('2024-11-05')
                print(sources[i])
                print(destinations[l])
                print(classes[j])
                driver.get(kayak)
                # Click "Show More" button
                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'ULvh-button.show-more-button')))
                '''count=2
                while count>1:
                    try:
                        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'ULvh-button.show-more-button'))).click()
                        print("MORE button clicked")
                    except TimeoutException:
                        break
                    count-=1
                    sleep(5)'''

                soup = BeautifulSoup(driver.page_source, 'html.parser')
                airlines = get_airline(soup)
                print(len(airlines))
                stops = get_stops(soup)
                print(len(stops))
                dep_time=get_dep_time(soup)
                print(len(dep_time))
                arr_time =get_arr_time(soup)
                print(len(arr_time))
                prices = get_price(soup)
                print(len(prices))
                duration = get_duration(soup)
                print(len(duration))
                df = pd.concat([df,pd.DataFrame({
                                'Airline': airlines,
                                'Duration': duration,
                                'stops' : stops,
                                'class' : classes[j],
                                'Source':sources[i],
                                'Destination': destinations[l],
                                'depature time' : dep_time ,
                                'arrival time' : arr_time,
                                'Price' : prices,
                                'Date' : '2024-11-05'})])


                df = df.replace('\n','', regex=True)
                #df = df.reset_index(drop = True)
                # save data as csv file for each route

                print(f"Succesfully saved {sources[i]} => {destinations[l]} route  ")
                driver.quit()

2024-11-05
CMN
PAR
economy
15
15
15
15
15
15
Succesfully saved CMN => PAR route  
2024-11-05
CMN
ROM
economy
15
15
15
15
15
15
Succesfully saved CMN => ROM route  
2024-11-05
CMN
IST
economy
15
15
15
15
15
15
Succesfully saved CMN => IST route  
2024-11-05
PAR
CMN
economy
15
15
15
15
15
15
Succesfully saved PAR => CMN route  
2024-11-05
PAR
ROM
economy
15
15
15
15
15
15
Succesfully saved PAR => ROM route  
2024-11-05
PAR
IST
economy
15
15
15
15
15
15
Succesfully saved PAR => IST route  
2024-11-05
ROM
CMN
economy
15
15
15
15
15
15
Succesfully saved ROM => CMN route  
2024-11-05
ROM
PAR
economy
15
15
15
15
15
15
Succesfully saved ROM => PAR route  
2024-11-05
ROM
IST
economy
15
15
15
15
15
15
Succesfully saved ROM => IST route  
2024-11-05
IST
CMN
economy
15
15
15
15
15
15
Succesfully saved IST => CMN route  
2024-11-05
IST
PAR
economy
15
15
15
15
15
15
Succesfully saved IST => PAR route  
2024-11-05
IST
ROM
economy
15
15
15
15
15
15
Succesfully saved IST => ROM route  
2024-11-05
CMN
P

In [14]:
df.to_csv('../data/airlines_dataset.csv',index=False)
driver.quit()

,Airline,Source,Destination,Duration,stops,class,depature time,arrival time,Price,Date
0,TUI Fly Belgium,CMN,PAR,3h 30m,direct,economy,07:15,10:45,AED 242,2023-12-09
1,Transavia France,CMN,PAR,3h 15m,direct,economy,19:25,22:40,AED 588,2023-12-09
2,Royal Air Maroc,CMN,PAR,2h 55m,direct,economy,18:30,21:25,AED 686,2023-12-09
3,Royal Air Maroc,CMN,PAR,3h 00m,direct,economy,13:25,16:25,AED 704,2023-12-09
4,Royal Air Maroc,CMN,PAR,2h 55m,direct,economy,16:15,19:10,AED 977,2023-12-09
...,...,...,...,...,...,...,...,...,...,...
10,British Airways,IST,ROM,10h 25m,1 stop,business,08:05,16:30,"AED 2,990",2024-11-05
11,British Airways,IST,ROM,13h 25m,1 stop,business,08:05,19:30,"AED 2,970",2024-11-05
12,TAROM,IST,ROM,14h 45m,1 stop,business,23:15,12:00+1,"AED 2,960",2024-11-05
13,KLM,IST,ROM,17h 15m,1 stop,business,17:55,09:10+1,"AED 2,840",2024-11-05
